### This file must be run on Colab with T4 GPU
Dataset : CNN Daily/Mail 3.0.0

Model : DistilBart

In [ ]:
!pip install --upgrade fsspec datasets

In [ ]:
# 사전 설치:
# pip install transformers datasets safetensors torch tqdm

import os
import torch
from tqdm import tqdm
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq
)
from torch.utils.data import DataLoader
from torch.optim import AdamW

# 1. 설정
model_name  = "sshleifer/distilbart-cnn-6-6"
cache_dir   = "./hf_cache"                   # 데이터셋·모델 캐시
save_dir    = "./distilbart_cnn_safetensors" # 최종 저장 경로
batch_size  = 2                              # 메모리에 따라 조정
num_epochs  = 1
max_samples = 2048                           # 전체 샘플 중 사용할 개수

# 2. 토크나이저·모델 로드
tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir=cache_dir)
model     = AutoModelForSeq2SeqLM.from_pretrained(model_name, cache_dir=cache_dir)

# 3. 데이터셋 로드 및 샘플 선택
raw_ds  = load_dataset("cnn_dailymail", "3.0.0", split="train", cache_dir=cache_dir)
dataset = raw_ds.select(range(max_samples))

# 4. 전처리 함수: encoder-decoder 구조이므로 "summarize:" 프롬프트 불필요
def preprocess(ex):
    enc = tokenizer(
        ex["article"],
        max_length=512,
        padding="max_length",
        truncation=True
    )
    lbl = tokenizer(
        ex["highlights"],
        max_length=128,
        padding="max_length",
        truncation=True
    )
    return {
        "input_ids":      enc["input_ids"],
        "attention_mask": enc["attention_mask"],
        "labels":         lbl["input_ids"]
    }

tokenized = dataset.map(preprocess, remove_columns=dataset.column_names)

# 5. DataLoader 구성
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
dataloader    = DataLoader(
    tokenized,
    batch_size=batch_size,
    shuffle=True,
    collate_fn=data_collator
)

# 6. 학습 준비 (기본은 GPU, 없으면 CPU)
device   = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
optimizer = AdamW(model.parameters(), lr=5e-5)

# 7. 학습 루프
model.train()
print("🚀 Training start")
for epoch in range(1, num_epochs+1):
    loop = tqdm(dataloader, desc=f"Epoch {epoch}")
    for step, batch in enumerate(loop):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if step % 5 == 0:
            loop.set_postfix(loss=f"{loss.item():.4f}")

print("✅ Training complete")

# 8. SafeTensors로 저장
os.makedirs(save_dir, exist_ok=True)
model.save_pretrained(save_dir, safe_serialization=True)
tokenizer.save_pretrained(save_dir)
print(f"✅ SafeTensors 모델이 '{save_dir}'에 저장되었습니다.")


In [ ]:
### 추후 학습 완료 된 model 을 가져올 때는 다음과 같이 진행하면 됨. 
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

save_dir = "./distilbart_cnn_safetensors"  # 학습 후 저장했던 경로
tokenizer = AutoTokenizer.from_pretrained(save_dir)
model     = AutoModelForSeq2SeqLM.from_pretrained(save_dir)

device = 0 if torch.cuda.is_available() else -1  # GPU:0, 없으면 CPU:-1

summarizer = pipeline(
    "summarization",
    model=model,
    tokenizer=tokenizer,
    device=device
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
### example article : 트럼프 관세 정책
article = """
US President Donald Trump has announced a series of tariffs, arguing they will boost American manufacturing and protect jobs.
But the import taxes have thrown the world economy into chaos and many have argued that they will make products more expensive for US consumers.
The US and China have now agreed to slash tariffs they had imposed on each other, for 90 days.
The UK and US have also announced a deal on tariffs, and other countries are hoping to reach an agreement with the White House.

What are tariffs and how do they work?
Tariffs are taxes charged on goods bought from other countries.
Typically, they are a percentage of a product's value.
A10% tariff means a $10 product would have a $1 tax on top - taking the total cost to $11 (£8.35).
Companies that bring foreign goods into the US have to pay the tax to the government.
They may pass some or all of the extra cost on to customers. Firms may also decide to import fewer goods.

Why is Trump using tariffs?
Trump says tariffs will encourage US consumers to buy more American-made goods, increase the amount of tax raised and lead to huge levels of investment.
He wants to reduce the gap between the value of goods the US buys from other countries and those it sells to them. He argues that America has been taken advantage of by "cheaters" and "pillaged" by foreigners.
The US president has made other demands alongside tariffs.
When he announced the first tariffs of his current term against China, Mexico and Canada, he said he wanted them to do more to stop migrants and illegal drugs reaching the US.
"""
summary = summarizer(
    article,
    max_length=60,    # 생성될 요약문의 최대 토큰 수
    min_length=20,    # 생성될 요약문의 최소 토큰 수
    do_sample=False   # 빔 서치(beam search) 사용
)[0]["summary_text"]

print("── 원문 ───────────────────────────────────────────")
print(article.strip(), "\n")
print("── 요약 ───────────────────────────────────────────")
print(summary)
